In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

gamma_opt = 1.50
beta_opt = 1.49

In [ ]:
# Build the QAOA circuit (copied from notebook 04)
edges = [(0,1), (0,2), (1,2), (1,3)]
n_qubits = 4


def build_qaoa_circuit(edges, n_qubits):
    qc = QuantumCircuit(n_qubits)
    gamma = Parameter('γ')
    beta = Parameter('β')

    for i in range(n_qubits):
        qc.h(i)

    for i, j in edges:
        qc.cx(i, j)
        qc.rz(2 * gamma, j)
        qc.cx(i, j)

    for i in range(n_qubits):
        qc.rx(2 * beta, i)

    qc.measure_all()
    return qc, gamma, beta


qc, gamma, beta = build_qaoa_circuit(edges, n_qubits)

In [3]:
backend = Aer.get_backend('qasm_simulator')

bound_qc = qc.assign_parameters({
    gamma: gamma_opt,
    beta: beta_opt
}, inplace=False)

t_qc = transpile(bound_qc, backend)
job = backend.run(t_qc, shots=4096)
result = job.result()
counts = result.get_counts()

counts

NameError: name 'qc' is not defined

In [ ]:
plot_histogram(counts)
plt.show()

In [ ]:
def bitstring_to_tuple(bitstring):
    return tuple(int(b) for b in bitstring[::-1])

results = []

for bitstring, count in counts.items():
    bits = bitstring_to_tuple(bitstring)
    cut = cut_value(bits, edges)
    results.append((bits, cut, count))

# Sort by frequency
results = sorted(results, key=lambda x: x[2], reverse=True)

results[:10]

In [ ]:
best_cut = max(r[1] for r in results)

best_solutions = [r for r in results if r[1] == best_cut]

best_cut, best_solutions